Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB3

Wrote a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, November 26 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, November 27
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1]:
from random import choices

import lab3_lib

In [2]:
fitness = lab3_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k = 50)
    print(''.join(str(g) for g in ind), ':', fitness(ind))

print(fitness.calls)

10101100011101100001010111101001011100010111010011 : 0.711115558
01001100010000111111000111000111110000010000111111 : 0.9115555578000001
11110101001000111001111110010100100110111101111000 : 0.7133333357999999
10110101000101001011101111111110100010101110111110 : 0.7111333556
10010110011110101011100000001101110010101001101111 : 0.7111355577999999
00001100010100101001110100001000000100111111101010 : 0.533355558
11010101100010011011011001011110000111101000101001 : 0.7133335578
00111010001101011000000111001010001010001011001101 : 0.7133557778
01101010010011011101101000000101111111101001111000 : 0.7133335557999999
10101101010100000000010101100101001101100011111010 : 0.7335555558
10


In [ ]:
class AgentTraining():

    class trainingType(Enum):
        DEFAULT = 0
        LAMBDA = 1
        #TOURNAMENT = 2

    def __init__(self, pop_size : int ) -> None:
        pass

    def init_population(self, size : int) -> None:
        print("Initializing population")
        for _ in range(size):
            temp = AgentGA()
            temp.compute_fitness(self.state)
            self.population.append(temp)

    def generation_default(self) -> None:
        for _ in range(len(self.population)):
            n = random.random()
            temp : Player = deepcopy(random.choice(self.population))
            if n > self.ration :
                temp.crossover(random.choice(self.population))
            else:
                temp.mutation(0.1)
            temp.compute_fitness(self.state)
            self.population.append(temp)    
        self.population.sort(key=lambda x : x.fitness , reverse=True)
        self.population = self.population[:len(self.population)//2]
        self.ration = 0.9 * self.ration

    def generation_lambda(self) -> None:
        parents = []
        # (mu,lambda) approach with lambda = mu for semplicity
        for _ in range(self.lam): # Selecting parents randomly
            parents.append(self.population.pop(random.randint(0,len(self.population)-1)) )
        for parent in parents:
            temp_population = []
            temp_population.append(deepcopy(parent))
            for _ in range(self.lam):
                temp : Player = deepcopy(parent).mutation(self.ration)
                temp.compute_fitness(self.state)
                temp_population.append(temp)  
            temp_population.sort(key=lambda x : x.fitness , reverse=True)
            self.population.append(temp_population[0])  
        self.population.sort(key=lambda x : x.fitness , reverse=True)
        self.ration = self.temperature * self.ration

    def train(self,epoch = 10):
        fitness = []
        avg_fitness = []
        print("Starting Training")
        for _ in range(epoch):
            print("Generation : ",_)
            self.generation()
            print(f"Best fitness : {self.population[0].fitness} , eval : {self.population[0].eval(self.state)}")
            fitness.append(self.population[0].fitness)
            avg_fitness.append(sum([x.fitness for x in self.population])/len(self.population))
        return self.population[0] , fitness , avg_fitness

In [ ]:
class Agent():
    def __init__(self) -> None:
        pass

    def compute_fitness(self) -> None:
        pass        


In [ ]:
class AgentGA(Agent):

    def __init__(self,rules : dict) -> None:
        self.rules = rules
        self.fitness = None

    def mutation(self,mutation_rate) -> Agent:
        for state in self.rules.keys():
            if random.random() < mutation_rate :
                self.rules[state] = random.choice([(r, o) for r, c in enumerate(state) for o in range(1, c + 1)])
        return self

    def crossover(self,other : Agent,other_weight = 0.5):
        for state in self.rules.keys():
            self.rules[state] = random.choices([self.rules[state],other.rules[state]],weights=[1-other_weight,other_weight])[0]

    def compute_fitness(self, state):
        pass

    
    def load_agent(self, path):
        with open(path, 'rb') as f:
            self.rules = pickle.load(f)
        return self
    
    def save_agent(self, path):
        with open(path, 'wb') as f:
            pickle.dump(self.rules, f)